# Day 22: Wizard Simulator 20XX

https://adventofcode.com/2015/day/22

In [195]:
def Match(spellSequence,hp=50,mana=500,bosshp=55,bossda=8,diff="E",verbose=True,spentMax=1000000):
    
    spellCost = {'M': 53, 'D': 73, 'S': 113, 'P': 173, 'R': 229}
    spellEffect = { "S": 0 , "P": 0 , "R": 0 } 
    armor = 0
    
    turn = 0 # counter for spell sequence
    wizardTurn = 1 # alternate between Wizard and Boss
    manaSpent = 0

    while True:
        
        if verbose:
            if wizardTurn==1:    
                print("-- Player --")
            else:
                print("-- Boss --")
                
        if wizardTurn==1 and diff=="H":
            hp -= 1   
            if verbose:
                print("Hard mode!")
                print("HP: {} - Mana {} - Boss HP {}".format(hp,mana,bosshp))
            if hp <= 0: # Wizard dies
                if verbose: print("Player dies!")
                return 0
        
        # Check residual effect of previous spells, if relevant apply. 
        # Apply to both Wizard and Boss turns.     
        if spellEffect['P']: # Poison
            if verbose: print("Poison!")
            spellEffect['P'] = max(spellEffect['P'] - 1, 0) 
            bosshp -= 3
        if spellEffect['S']: # Shield
            if verbose: print("Shield!")
            spellEffect['S'] = max(spellEffect['S'] - 1, 0)
            armor = 7
        else:
            armor = 0        
        if spellEffect['R']: # Recharge
            if verbose: print("Recharge!")
            spellEffect['R'] = max(spellEffect['R'] - 1, 0)
            mana += 101
        
        if bosshp <= 0:
            if verbose: print("Boss dies!")
            return manaSpent
        
        # Wizard turn
        if wizardTurn==1:
            
            if turn>=len(spellSequence):
                if verbose:
                    print("Reached end of spell sequence. Exiting.")
                return 0
                
            spell = spellSequence[turn]
            mana -= spellCost[spell]
            if mana<0: # not enough mana to cast the spell
                if verbose: 
                    print("Not enough mana to cast spell",spell,". Exiting.")
                return 0
            manaSpent += spellCost[spell]
            if manaSpent >= spentMax:
                if verbose: 
                    print("Spent more mana than maximum allowed. Exiting.")
                    return 0

            if spell == 'M':
                if verbose: print("Missile!")
                bosshp -= 4
            elif spell == 'D':
                if verbose: print("Defense!")
                bosshp -= 2
                hp += 2
            else:
                if spellEffect[spell]:
                    if verbose: print("Spell sequence not allowed. Exiting.")
                    return 0 # sequence not allowed
                else:
                    if verbose: print("Casting spell",spell,". Will be active from next turn.")
                    if spell=='S':
                        spellEffect['S'] = 6
                    elif spell == 'P':
                        spellEffect['P'] = 6
                    elif spell=='R':
                        spellEffect['R'] = 5

            if verbose:
                print("HP: {} - Mana {} - Boss HP {}".format(hp,mana,bosshp))
            
            if bosshp <= 0:
                if verbose: print("Boss dies!")
                return manaSpent

        # Boss turn
        else:
            #print(hp,mana,bosshp)
            hp -= max(bossda - armor, 1)
            
            if verbose:
                print("HP: {} - Mana {} - Boss HP {}".format(hp,mana,bosshp))
            
            if hp <= 0: # Wizard dies
                if verbose: print("Player dies!")
                return 0
        
        # increment turn counter only once per Wizard + Boss turn
        if wizardTurn == 1:
            turn += 1
        
        wizardTurn = -wizardTurn # flip from -1 to 1

In [196]:
Match(["P","M"],10,250,13,8)

-- Player --
Casting spell P . Will be active from next turn.
HP: 10 - Mana 77 - Boss HP 13
-- Boss --
Poison!
HP: 2 - Mana 77 - Boss HP 10
-- Player --
Poison!
Missile!
HP: 2 - Mana 24 - Boss HP 3
-- Boss --
Poison!
Boss dies!


226

In [197]:
Match(["R","S","D","P","M"],10,250,14,8)

-- Player --
Casting spell R . Will be active from next turn.
HP: 10 - Mana 21 - Boss HP 14
-- Boss --
Recharge!
HP: 2 - Mana 122 - Boss HP 14
-- Player --
Recharge!
Casting spell S . Will be active from next turn.
HP: 2 - Mana 110 - Boss HP 14
-- Boss --
Shield!
Recharge!
HP: 1 - Mana 211 - Boss HP 14
-- Player --
Shield!
Recharge!
Defense!
HP: 3 - Mana 239 - Boss HP 12
-- Boss --
Shield!
Recharge!
HP: 2 - Mana 340 - Boss HP 12
-- Player --
Shield!
Casting spell P . Will be active from next turn.
HP: 2 - Mana 167 - Boss HP 12
-- Boss --
Poison!
Shield!
HP: 1 - Mana 167 - Boss HP 9
-- Player --
Poison!
Shield!
Missile!
HP: 1 - Mana 114 - Boss HP 2
-- Boss --
Poison!
Boss dies!


641

In [198]:
# generate random sequence of spells of aribitrary lenght

from random import choice

spells = ["R","S","D","P","M"]

def randomSpellSequence(length):
    return [ choice(spells) for _ in range(length) ]

In [209]:
spentMin = 10000
spentMinSeq = []
checkTail = 4

while True:
    
    spellSequence = randomSpellSequence(20)
    manaSpent = Match(spellSequence,50,500,55,8,diff="E",verbose=False,spentMax=spentMin)
    
    if manaSpent:# player wins
        if manaSpent <= spentMin:
            spentMin = manaSpent
            spentMinSeq.append(spentMin)
            print(manaSpent,spellSequence)
            if len(spentMinSeq)>checkTail:
                if sum(spentMinSeq[-checkTail:])/checkTail == spentMinSeq[-1]:
                    break
                    
print("Asymptotic minimum mana Part 1 =",spentMinSeq[-1])

1953 ['R', 'P', 'S', 'D', 'P', 'R', 'D', 'S', 'R', 'D', 'P', 'R', 'D', 'M', 'M', 'R', 'D', 'R', 'R', 'P']
1518 ['P', 'R', 'D', 'S', 'P', 'R', 'M', 'P', 'R', 'D', 'R', 'R', 'R', 'S', 'P', 'P', 'D', 'M', 'D', 'D']
1455 ['R', 'S', 'P', 'M', 'R', 'S', 'P', 'D', 'D', 'P', 'M', 'R', 'R', 'S', 'M', 'S', 'R', 'R', 'P', 'D']
1382 ['R', 'S', 'P', 'R', 'M', 'P', 'S', 'D', 'P', 'M', 'M', 'R', 'M', 'M', 'S', 'M', 'R', 'M', 'M', 'S']
1295 ['S', 'R', 'P', 'M', 'R', 'P', 'S', 'M', 'M', 'M', 'M', 'R', 'D', 'R', 'D', 'P', 'S', 'M', 'M', 'M']
1289 ['P', 'R', 'D', 'P', 'R', 'S', 'P', 'M', 'D', 'M', 'R', 'S', 'D', 'M', 'S', 'R', 'P', 'D', 'R', 'S']
1269 ['P', 'M', 'R', 'D', 'P', 'R', 'S', 'P', 'M', 'R', 'R', 'P', 'P', 'D', 'S', 'S', 'R', 'M', 'D', 'R']
953 ['R', 'P', 'M', 'M', 'S', 'P', 'M', 'M', 'M', 'D', 'D', 'S', 'S', 'R', 'P', 'M', 'S', 'S', 'D', 'P']
953 ['M', 'P', 'R', 'S', 'P', 'M', 'M', 'M', 'M', 'M', 'S', 'D', 'S', 'D', 'P', 'P', 'D', 'M', 'P', 'S']
953 ['R', 'M', 'P', 'S', 'M', 'P', 'M', 'M', 'M'

In [208]:
spentMin = 10000
spentMinSeq = []
checkTail = 5

while True:
    
    spellSequence = randomSpellSequence(20)
    manaSpent = Match(spellSequence,50,500,55,8,diff="H",verbose=False,spentMax=spentMin)
    
    if manaSpent:# player wins
        if manaSpent <= spentMin:
            spentMin = manaSpent
            spentMinSeq.append(spentMin)
            print(manaSpent,spellSequence)
            if len(spentMinSeq)>checkTail:
                if sum(spentMinSeq[-checkTail:])/checkTail == spentMinSeq[-1]:
                    break

print("Asymptotic minimum mana Part 2 =",spentMinSeq[-1])

1475 ['R', 'M', 'S', 'P', 'R', 'S', 'P', 'M', 'M', 'P', 'S', 'S', 'M', 'D', 'S', 'S', 'D', 'P', 'P', 'M']
1382 ['R', 'P', 'M', 'R', 'S', 'P', 'D', 'S', 'P', 'M', 'R', 'P', 'R', 'P', 'S', 'M', 'D', 'S', 'D', 'D']
1362 ['R', 'P', 'S', 'R', 'M', 'P', 'M', 'S', 'P', 'M', 'D', 'S', 'M', 'M', 'P', 'R', 'S', 'D', 'S', 'P']
1362 ['M', 'R', 'P', 'S', 'R', 'P', 'M', 'S', 'P', 'M', 'S', 'R', 'R', 'D', 'P', 'D', 'P', 'P', 'P', 'D']
1309 ['R', 'P', 'M', 'S', 'P', 'R', 'S', 'P', 'M', 'D', 'M', 'P', 'S', 'R', 'P', 'M', 'S', 'S', 'M', 'S']
1309 ['P', 'R', 'S', 'P', 'M', 'R', 'P', 'S', 'M', 'D', 'P', 'M', 'M', 'R', 'R', 'P', 'S', 'P', 'S', 'P']
1309 ['P', 'M', 'R', 'S', 'P', 'R', 'S', 'P', 'M', 'D', 'P', 'S', 'M', 'R', 'S', 'M', 'M', 'P', 'D', 'R']
1309 ['P', 'R', 'M', 'P', 'S', 'R', 'P', 'S', 'M', 'P', 'S', 'M', 'S', 'P', 'S', 'S', 'D', 'P', 'S', 'D']
1289 ['P', 'R', 'S', 'P', 'R', 'D', 'P', 'D', 'M', 'S', 'S', 'M', 'M', 'M', 'R', 'M', 'M', 'S', 'M', 'D']
1289 ['P', 'R', 'S', 'P', 'R', 'D', 'P', 'D', 